# AIM: Extract features on labeled data using the pretrained EEGNet
across subject pretext task

In [1]:
import numpy as np
import pandas as pd
import mne
import lightning.pytorch as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
from torchmetrics import F1Score, Accuracy
import random
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedGroupKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from IPython.display import display
%matplotlib inline


# prevent extensive logging
mne.set_log_level('WARNING')

## Loading epoch data & participant data of labeled sample
These dataframes have been filtered and stored in a previous project. See https://github.com/TSmolders/Internship_EEG for original code

In [2]:

df_participants = pd.read_pickle(r'D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TDBRAIN_participants_V2_data\df_participants.pkl')
sample_df = pd.read_pickle(r'D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TD-BRAIN_extracted_features\df_selected_stat_features.pkl')
sample_ids = sample_df['ID'].unique() # obtain unique IDs from subsampled dataframe containing epoched features
df_sample = df_participants[df_participants['participants_ID'].isin(sample_ids)] # filter participants dataframe to only include subsampled IDs
df_sample = df_sample[df_sample['sessID'] == 1] # filter first session
print(df_sample.shape)
print(df_sample['diagnosis'].value_counts())



(225, 12)
diagnosis
ADHD       45
HEALTHY    45
MDD        45
OCD        45
SMC        45
Name: count, dtype: int64


In [3]:
# functions for loading the epoched EEG data
def get_filepath(epoch_dir, participant_ids):
    """
    Function to get the filepath of the epoched EEG recording
    :param epoch_dir: directory containing the epoched EEG recordings
    :param participant_ids: list of participant IDs to include
    """
    filepaths = []
    found_ids = set()
    
    for subdir, dirs, files in os.walk(epoch_dir):
        for file in files:
            for participant_id in participant_ids:
                if participant_id in file:
                    filepaths.append(os.path.join(subdir, file))
                    found_ids.add(participant_id)
    
    # Print participant IDs if no files matching those IDs are found
    for participant_id in participant_ids:
        if participant_id not in found_ids:
            print(f"No files found for participant ID {participant_id}")

    return filepaths

class EpochDataset(torch.utils.data.Dataset):
    def __init__(self, participant_ids, epoch_dir):
        self.filepaths = get_filepath(epoch_dir, participant_ids)
        self.participant_ids = participant_ids
        self.epochs = []
        self.participant_ids = []
        self._load_data()
        print(f"Number of epochs: {self.epochs.shape[0]}")
        print(f"Number of participants: {len(self.participant_ids)}")

    def _load_data(self):
        all_epochs = []
        for filepath in self.filepaths:
            epochs = torch.load(filepath)
            # get participant ID from filepath to make sure the participant ID is correct
            participant_id = filepath.split("\\")[-1].split(".")[0]
            all_epochs.append(epochs)
            self.participant_ids.extend([participant_id]*epochs.shape[0])
        self.epochs = np.concatenate(all_epochs, axis=0)

    def __len__(self):
        return self.epochs.shape[0]
    
    def __getitem__(self, idx):
        epoch = self.epochs[idx]
        participant_id = self.participant_ids[idx]
        return torch.tensor(epoch, dtype=torch.float32), participant_id

In [4]:
# load the epochs into a dataset
participant_ids = df_sample['participants_ID'].tolist()
dataset = EpochDataset(participant_ids, r"D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TDBRAIN-dataset-derivatives\thesis_epoched_data\EC")
print(len(dataset))
print(dataset[0][0].shape)
print(dataset[0][1])
print(dataset[1][1])

No files found for participant ID sub-88073521
Number of epochs: 2688
Number of participants: 2688
2688
torch.Size([26, 1244])
sub-87964717
sub-87964717


participant sub-88073521 EC's recording is marked as BAD by the preprocessing pipeline of TDBRAIN

## Transfering pretrained weights & extracting features

### Functions:

In [5]:
def transfer_weights(pretrained_weights, pretext_model):
    """
    Function to transfer the pretrained weights to the pretext model
    param: pretrained_weights: the weights to transfer in a dictionary
    param: pretext_model: the model to transfer the weights to
    """
    pretrained_model = pretext_model
    modified_keys = {}
    for k, v in pretrained_weights.items():
        decomposed_key = k.split('.')
        if decomposed_key[0] == 'EEGNet' or decomposed_key[0] == 'ShallowNet': # remove the first part of each key to match the model's keys
            pretrained_key = '.'.join(decomposed_key[1:])
            modified_keys[pretrained_key] = v

            
    pretrained_model.load_state_dict(modified_keys)
        
    return pretrained_model

def extract_features(pretrained_model, data, pretext_task, df_sample, to_disk=False, num_extracted_features=100, add_missing_ids=True):
    """
    Function to extract features from the pretrained model
    param: pretrained_model: the model to extract features from
    param: data: the dataset containing the epochs to extract features from
    param: pretext_task: a string indicating the specific pretext task to save the features as
    param: df_sample: the dataframe containing the sampled participant IDs and their corresponding diagnosis
    param: to_disk: boolean to save the features to disk
    param: num_extracted_features: the number of features to extract
    param: add_missing_ids: boolean to add missing participant IDs to the dataframe
    """
    dataloader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False)
    pretrained_model.eval()
    features_list = []
    participant_ids = []
    with torch.no_grad():  # Disable gradient calculation
        for batch in dataloader:
            epoch, participant_id = batch  # Remove the batch dimension
            epoch = epoch.unsqueeze(0)  # Add dimension
            # print(epoch.shape)
            features = pretrained_model(epoch)  # Extract features
            features = features.squeeze(0)
            features = features.numpy()
            features_list.append(features)
            participant_ids.append(participant_id[0])

    if add_missing_ids:
        # Probably not the best idea to add missing IDs with NaN values (in this
        # case only one ID), but the baselines from the previous internetship were 
        # trained with the same approach (using mean impute for the 12 missing epochs of one participant). 
        # Considering only 1 participant is missing (due to 1 BAD preprocessing file), 
        # it shouldn't have a large impact on the results. But it's something to keep in mind for future work.

        # check if any IDs are missing compared to the sampled IDs
        missing_ids = set(df_sample['participants_ID'].tolist()) - set(participant_ids)
        # if they are missing, add them with NaN values
        if missing_ids:
            for missing_id in missing_ids:
                num_missing_epochs = 12  # Assuming 12 missing epochs for each missing participant
                for _ in range(num_missing_epochs):
                    features_list.append([np.nan] * num_extracted_features)
                    participant_ids.append(missing_id)

    features_df = pd.DataFrame(features_list) # store as dataframe
    features_df['ID'] = participant_ids # add participant IDs to the dataframe

    # map the diagnosis values from df_sample to the dataframe based on participant IDs
    features_df['diagnosis'] = features_df['ID'].map(df_sample.set_index('participants_ID')['diagnosis'])

    
    print(f'{features_df.shape = }')
    # print(f'{features_df["diagnosis"].value_counts()}')
    # print(f'{features_df.isnull().sum()}')
    display(features_df.head(3))


    if to_disk:
        features_df.to_pickle(f'D:/Documents/Master_Data_Science/Thesis/thesis_code/DataScience_Thesis/data/SSL_features/df_{pretext_task}_features.pkl')
    
    return features_df

def evaluate_features(features_df):
    """
    Function to quickly evaluate the extracted features. Doesn't stratify/group data splitting! 
    """
    groups = features_df['ID']
    X = features_df.drop(['ID', 'diagnosis'], axis=1)
    y = features_df['diagnosis']

    # if X contains NaN values, impute them with the feature-wise mean
    if X.isnull().values.any():
        X = X.fillna(X.mean())

    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=False)

    # Get the train and test indices
    for train_index, test_index in sgkf.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        break  # Only use the first split
    
    # quick SVM model
    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('quick SVM model')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(f1_score(y_test, y_pred, average='macro'))
    print()

    # quick random forest model
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('quick random forest model')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(f1_score(y_test, y_pred, average='macro'))

    return

def get_ssl_features(
        pretext_task,
        data,
        df_sample,
        num_extracted_features=100,
        eval=True,
        to_disk=True,
        pretext_model='EEGNet',
        conv_length=25,
        n_filters=40,
        weights_dir=r'D:\Documents\Master_Data_Science\Thesis\thesis_code\DataScience_Thesis\data\pretext_model_weights'
                     ):
    """
    Obtains SSL features from the weights trained by the pretext model.
    param: pretext_task: a string indicating the specific pretext task to load the weights from and save the features as
    param: data: the dataset containing the epochs to extract features from
    param: df_sample: the dataframe containing the sampled participant IDs and their corresponding diagnosis
    param: num_extracted_features: the number of features to extract
    param: eval: boolean to evaluate the features
    param: to_disk: boolean to save the features to disk
    param: pretext_model: the model to extract features from
    param: weights_dir: the directory containing the weights of the pretext model
    """
    # Need to define model class here to avoid issues with different number of extracted features
    # create Conv2d with max norm constraint
    class Conv2dWithConstraint(nn.Conv2d):
        def __init__(self, *args, max_norm: int = 1, **kwargs):
            self.max_norm = max_norm
            super(Conv2dWithConstraint, self).__init__(*args, **kwargs)

        def forward(self, x: torch.Tensor) -> torch.Tensor:
            self.weight.data = torch.renorm(self.weight.data, p=2, dim=0, maxnorm=self.max_norm)
            return super(Conv2dWithConstraint, self).forward(x)
        
    class EEGNet(nn.Module):
        """
        Code taken and adjusted from pytorch implementation of EEGNet
        url: https://github.com/torcheeg/torcheeg/blob/v1.1.0/torcheeg/models/cnn/eegnet.py#L5
        """
        def __init__(self,
                    chunk_size: int = 1244, # number of data points in each EEG chunk
                    num_electrodes: int = 26, # number of EEG electrodes
                    F1: int = 8, # number of filters in first convolutional layer
                    F2: int = 16, # number of filters in second convolutional layer
                    D: int = 2, # depth multiplier
                    num_extracted_features: int = num_extracted_features, # number of features to extract
                    kernel_1: int = 64, # the filter size of block 1 (half of sfreq (125 Hz))
                    kernel_2: int = 16, # the filter size of block 2 (one eight of sfreq (500 Hz))
                    dropout: float = 0.25): # dropout rate
            super(EEGNet, self).__init__()
            self.F1 = F1
            self.F2 = F2
            self.D = D
            self.chunk_size = chunk_size
            self.num_extracted_features = num_extracted_features
            self.num_electrodes = num_electrodes
            self.kernel_1 = kernel_1
            self.kernel_2 = kernel_2
            self.dropout = dropout

            self.block1 = nn.Sequential(
                nn.Conv2d(1, self.F1, (1, self.kernel_1), stride=1, padding=(0, self.kernel_1 // 2), bias=False),
                nn.BatchNorm2d(self.F1, momentum=0.01, affine=True, eps=1e-3),
                Conv2dWithConstraint(self.F1,
                                    self.F1 * self.D, (self.num_electrodes, 1),
                                    max_norm=1,
                                    stride=1,
                                    padding=(0, 0),
                                    groups=self.F1,
                                    bias=False), nn.BatchNorm2d(self.F1 * self.D, momentum=0.01, affine=True, eps=1e-3),
                nn.ELU(), nn.AvgPool2d((1, 4), stride=4), nn.Dropout(p=dropout))

            self.block2 = nn.Sequential(
                nn.Conv2d(self.F1 * self.D,
                        self.F1 * self.D, (1, self.kernel_2),
                        stride=1,
                        padding=(0, self.kernel_2 // 2),
                        bias=False,
                        groups=self.F1 * self.D),
                nn.Conv2d(self.F1 * self.D, self.F2, 1, padding=(0, 0), groups=1, bias=False, stride=1),
                nn.BatchNorm2d(self.F2, momentum=0.01, affine=True, eps=1e-3), nn.ELU(), nn.AvgPool2d((1, 8), stride=8),
                nn.Dropout(p=dropout))

            self.lin = nn.Linear(self.feature_dim(), num_extracted_features, bias=False)


        def feature_dim(self):
            # function to calculate the number of features after the convolutional blocks
            with torch.no_grad():
                mock_eeg = torch.zeros(1, 1, self.num_electrodes, self.chunk_size)

                mock_eeg = self.block1(mock_eeg)
                mock_eeg = self.block2(mock_eeg)

            return self.F2 * mock_eeg.shape[3]

        def forward(self, x: torch.Tensor) -> torch.Tensor:
            x = self.block1(x)
            x = self.block2(x)
            x = x.flatten(start_dim=1)
            x = self.lin(x)
            return x
    
    class ShallowNet(nn.Module):
        """
        Pytorch implementation of the ShallowNet Encoder.
        Code taken and adjusted from:
        https://github.com/MedMaxLab/selfEEG/blob/024402ba4bde95051d86ab2524cc71105bfd5c25/selfeeg/models/zoo.py#L693
        """

        def __init__(self,
                    samples=1244,
                    chans=26, # number of EEG channels
                    F=n_filters, # number of output filters in the temporal convolution layer
                    K1=conv_length, # length of the temporal convolutional layer
                    pool=75, # temporal pooling kernel size
                    dropout=0.2, # dropout probability
                    num_extracted_features=num_extracted_features # number of features to extract
                    ):

            super(ShallowNet, self).__init__()
            self.conv1 = nn.Conv2d(1, F, (1, K1), stride=(1, 1))
            self.conv2 = nn.Conv2d(F, F, (chans, 1), stride=(1, 1))
            self.batch1 = nn.BatchNorm2d(F)
            self.pool2 = nn.AvgPool2d((1, pool), stride=(1, 15))
            self.flatten2 = nn.Flatten()
            self.drop1 = nn.Dropout(dropout)
            self.lin = nn.Linear(
                F * ((samples - K1 + 1 - pool) // 15 + 1), num_extracted_features
            )

        def forward(self, x):
            x = self.conv1(x)
            x = self.conv2(x)
            x = self.batch1(x)
            x = torch.square(x)
            x = self.pool2(x)
            x = torch.log(torch.clamp(x, 1e-7, 10000))
            x = self.flatten2(x)
            x = self.drop1(x)
            x = self.lin(x)

            return x
    
    if pretext_model == 'EEGNet':
        pretext_model = EEGNet()
    if pretext_model == 'ShallowNet':
        pretext_model = ShallowNet()
        
    pretrained_weights = torch.load(f'{weights_dir}\{pretext_task}_weights.pt')
    pretrained_model = transfer_weights(pretrained_weights, pretext_model)
    features_df = extract_features(pretrained_model, data, pretext_task, df_sample=df_sample, to_disk=to_disk, num_extracted_features=num_extracted_features)
    if eval:
        evaluate_features(features_df)
    
    return

### 0. randomly initialized model

In [7]:
features_df = extract_features(EEGNet(), dataset, df_sample=df_sample, pretext_task='random', to_disk=False)
evaluate_features(features_df)

features_df.shape = (2688, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-0.025231,0.558265,0.386430,-0.427448,-0.195549,-0.385354,0.035971,0.385304,0.106288,0.573986,...,0.261263,0.668248,-0.518208,0.242272,0.186938,0.214983,0.118263,-0.321973,sub-87964717,SMC
1,-0.111984,0.977839,0.131731,-0.051678,-0.290045,-0.434035,0.237411,-0.101681,-0.275212,0.762277,...,0.520117,0.624576,-0.485054,0.251544,-0.037080,-0.125105,-0.325259,-0.672581,sub-87964717,SMC
2,-0.033515,0.690026,-0.037421,-0.427968,-0.044509,-0.273484,0.223849,0.226518,0.001657,0.772421,...,0.178688,0.269243,-0.326347,-0.047144,-0.056313,0.331418,0.099657,-0.459115,sub-87964717,SMC


quick SVM model
[[58 17  5 16 12]
 [15 36  9 32 16]
 [13 29 24 21 21]
 [ 0 27 21 33 27]
 [ 3  4 15 26 60]]
              precision    recall  f1-score   support

        ADHD       0.65      0.54      0.59       108
     HEALTHY       0.32      0.33      0.33       108
         MDD       0.32      0.22      0.26       108
         OCD       0.26      0.31      0.28       108
         SMC       0.44      0.56      0.49       108

    accuracy                           0.39       540
   macro avg       0.40      0.39      0.39       540
weighted avg       0.40      0.39      0.39       540

0.3899649803774411

quick random forest model
[[62 14 14 10  8]
 [18 34 20 22 14]
 [22 28 26 15 17]
 [10 24 28 18 28]
 [ 3 23 15 19 48]]
              precision    recall  f1-score   support

        ADHD       0.54      0.57      0.56       108
     HEALTHY       0.28      0.31      0.29       108
         MDD       0.25      0.24      0.25       108
         OCD       0.21      0.17      0.19       

### 3. AcrossSub pretext task

#### 3a. Default (25 length)

In [6]:
# best model checkpoint
get_ssl_features('acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-0.813289,-2.716501,0.919075,0.711839,-0.579521,-0.587127,-1.335802,0.153440,-0.412433,-2.323945,...,0.507714,2.636543,0.503704,0.827685,0.395858,0.120876,-0.415438,-0.451464,sub-87964717,SMC
1,-0.504195,-2.467970,1.108659,1.404283,0.195515,-2.225784,-2.613005,-0.281208,-0.971093,-2.152495,...,-1.801580,3.331660,1.010935,1.049690,2.478003,0.066149,-0.315350,-0.660595,sub-87964717,SMC
2,-0.830035,-2.191188,0.291836,1.731114,0.140483,-1.779407,-1.535045,-0.424333,0.150285,-1.432060,...,0.596767,2.368657,-0.707786,0.529840,1.082571,-0.320324,0.054474,-0.413685,sub-87964717,SMC


quick SVM model
[[55 12 12 24  5]
 [ 1 50  2 40 15]
 [ 0 36 16  8 48]
 [ 3 39 13 37 16]
 [ 0 18 26 11 53]]
              precision    recall  f1-score   support

        ADHD       0.93      0.51      0.66       108
     HEALTHY       0.32      0.46      0.38       108
         MDD       0.23      0.15      0.18       108
         OCD       0.31      0.34      0.32       108
         SMC       0.39      0.49      0.43       108

    accuracy                           0.39       540
   macro avg       0.44      0.39      0.40       540
weighted avg       0.44      0.39      0.40       540

0.3953832393595814

quick random forest model
[[56 14  8 28  2]
 [ 3 44 12 43  6]
 [12 25 22  5 44]
 [12 28 27 23 18]
 [ 5 10 18 15 60]]
              precision    recall  f1-score   support

        ADHD       0.64      0.52      0.57       108
     HEALTHY       0.36      0.41      0.38       108
         MDD       0.25      0.20      0.23       108
         OCD       0.20      0.21      0.21       

In [7]:
# overtrained model
get_ssl_features('fullytrained_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet')

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-33.949890,-21.187607,40.104279,4.188790,-12.426004,-3.792988,-6.778365,3.670086,-8.454637,-12.776423,...,9.429151,48.784637,-5.315949,17.138603,8.304023,-19.232035,20.425667,-27.241877,sub-87964717,SMC
1,-35.625240,-20.766567,44.157349,2.638543,-7.202231,3.174234,-19.622866,-10.128608,-2.209416,-7.286963,...,-3.001688,52.417896,-4.882135,18.492725,-2.724310,-12.108669,22.986965,-19.918627,sub-87964717,SMC
2,-36.063278,-19.276077,38.083809,-1.578408,2.942086,-7.001598,-11.447214,0.169034,3.982788,-9.969617,...,12.822247,45.334496,-4.750672,25.247160,6.399560,-13.530219,19.931404,-23.771576,sub-87964717,SMC


quick SVM model
[[60  0 32 14  2]
 [12 53  3 40  0]
 [ 7 23 21 34 23]
 [45  8 12 42  1]
 [ 3 18 32  0 55]]
              precision    recall  f1-score   support

        ADHD       0.47      0.56      0.51       108
     HEALTHY       0.52      0.49      0.50       108
         MDD       0.21      0.19      0.20       108
         OCD       0.32      0.39      0.35       108
         SMC       0.68      0.51      0.58       108

    accuracy                           0.43       540
   macro avg       0.44      0.43      0.43       540
weighted avg       0.44      0.43      0.43       540

0.43045500760769845

quick random forest model
[[60  2 34 12  0]
 [ 5 59  3 41  0]
 [ 9 23 10 37 29]
 [29 14 17 45  3]
 [ 0 10 40  0 58]]
              precision    recall  f1-score   support

        ADHD       0.58      0.56      0.57       108
     HEALTHY       0.55      0.55      0.55       108
         MDD       0.10      0.09      0.09       108
         OCD       0.33      0.42      0.37      

#### 3b. 50 length

In [8]:
# best model checkpoint
get_ssl_features('50kernel_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet',
                 conv_length=50)

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-2.458650,-0.548142,1.083744,0.146970,-2.763436,-0.161980,-0.095017,-0.401116,-0.851674,0.050277,...,-1.988437,0.882748,-0.111203,0.647702,0.166173,0.700321,0.649734,0.613534,sub-87964717,SMC
1,0.265251,-3.939014,-1.821749,1.061366,-0.764300,3.637352,-1.261581,-0.842359,1.410532,-2.138444,...,-1.080681,1.009907,-0.775154,0.628145,1.104165,-2.414675,0.255057,0.679903,sub-87964717,SMC
2,-1.688862,-0.581559,-0.203498,-0.067555,0.869881,0.197307,-1.429087,-0.664695,0.178362,-0.972831,...,2.062059,3.545937,-0.342452,0.758872,3.350813,-0.766527,-0.744713,-0.165280,sub-87964717,SMC


quick SVM model
[[60  8 13 19  8]
 [ 0 48  9 20 31]
 [ 1 18 10 24 55]
 [ 0 22 40  6 40]
 [ 4 32 22 18 32]]
              precision    recall  f1-score   support

        ADHD       0.92      0.56      0.69       108
     HEALTHY       0.38      0.44      0.41       108
         MDD       0.11      0.09      0.10       108
         OCD       0.07      0.06      0.06       108
         SMC       0.19      0.30      0.23       108

    accuracy                           0.29       540
   macro avg       0.33      0.29      0.30       540
weighted avg       0.33      0.29      0.30       540

0.29890925687567715

quick random forest model
[[59 14  2 29  4]
 [ 4 42 10 33 19]
 [10 38 17 16 27]
 [ 7 24 41 16 20]
 [12 17 29 18 32]]
              precision    recall  f1-score   support

        ADHD       0.64      0.55      0.59       108
     HEALTHY       0.31      0.39      0.35       108
         MDD       0.17      0.16      0.16       108
         OCD       0.14      0.15      0.15      

In [9]:
# overtrained model
get_ssl_features('fullytrained_50kernel_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet',
                 conv_length=50)

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,2.503716,-3.903119,0.817824,-7.813189,-4.968616,-0.979354,-0.792452,-3.209583,-11.079519,0.492422,...,-10.436070,-1.252131,14.167785,23.144547,0.428988,-40.334454,2.021379,-2.786710,sub-87964717,SMC
1,5.927292,5.149239,0.351382,-10.125467,-2.937922,-1.268776,-1.432976,-4.310213,-9.371466,-6.774857,...,-4.930865,-7.551412,13.762752,18.957684,1.831605,-37.478588,1.447488,-3.028751,sub-87964717,SMC
2,-4.624651,-2.265534,3.518987,-10.885687,-2.918881,-1.505024,-2.785959,-0.440287,-6.684763,-4.026264,...,-3.798706,-1.087756,12.991506,19.899950,2.009175,-37.491917,0.907675,-4.576303,sub-87964717,SMC


quick SVM model
[[52  0 32 20  4]
 [ 8 49 11 39  1]
 [10 21 31 24 22]
 [35  3 44 16 10]
 [ 3 17 30  3 55]]
              precision    recall  f1-score   support

        ADHD       0.48      0.48      0.48       108
     HEALTHY       0.54      0.45      0.49       108
         MDD       0.21      0.29      0.24       108
         OCD       0.16      0.15      0.15       108
         SMC       0.60      0.51      0.55       108

    accuracy                           0.38       540
   macro avg       0.40      0.38      0.38       540
weighted avg       0.40      0.38      0.38       540

0.3841998857623858

quick random forest model
[[59  4 27 16  2]
 [ 6 61 10 31  0]
 [13 38 18 15 24]
 [38 12 35 12 11]
 [ 1 13 42  0 52]]
              precision    recall  f1-score   support

        ADHD       0.50      0.55      0.52       108
     HEALTHY       0.48      0.56      0.52       108
         MDD       0.14      0.17      0.15       108
         OCD       0.16      0.11      0.13       

#### 3c. 75 length

In [10]:
# best model checkpoint
get_ssl_features('75kernel_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet',
                 conv_length=75)

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,0.840859,0.590983,0.467424,3.076359,0.036218,1.402541,-2.051340,-2.824054,0.628831,2.102428,...,1.792808,0.392887,1.841016,5.426975,-1.137669,-1.313870,2.199677,3.258967,sub-87964717,SMC
1,2.858373,1.471146,-1.137222,3.663265,-1.513606,2.690763,-2.161195,-3.813105,2.241127,3.074813,...,0.329486,0.154818,2.998334,3.021048,0.467583,0.586876,2.969292,1.756182,sub-87964717,SMC
2,1.661513,0.128069,-0.356677,2.382121,-0.494481,1.930204,5.477932,-2.530767,1.595502,1.648758,...,1.330771,-1.320557,2.186252,2.934978,0.195836,-0.028694,2.636691,1.778278,sub-87964717,SMC


quick SVM model
[[58 10 14 25  1]
 [ 0 32 10 46 20]
 [ 0 19  1 53 35]
 [ 0 23 22 32 31]
 [ 1  5 33 45 24]]
              precision    recall  f1-score   support

        ADHD       0.98      0.54      0.69       108
     HEALTHY       0.36      0.30      0.32       108
         MDD       0.01      0.01      0.01       108
         OCD       0.16      0.30      0.21       108
         SMC       0.22      0.22      0.22       108

    accuracy                           0.27       540
   macro avg       0.35      0.27      0.29       540
weighted avg       0.35      0.27      0.29       540

0.29128399921085185

quick random forest model
[[58  8 24 14  4]
 [12 28 18 24 26]
 [ 9 18 30 19 32]
 [15 30 25 20 18]
 [ 9 23 26 14 36]]
              precision    recall  f1-score   support

        ADHD       0.56      0.54      0.55       108
     HEALTHY       0.26      0.26      0.26       108
         MDD       0.24      0.28      0.26       108
         OCD       0.22      0.19      0.20      

In [11]:
# overtrained model
get_ssl_features('fullytrained_75kernel_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet',
                 conv_length=75)

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,12.691360,5.094767,32.233116,15.758214,-1.342315,-0.590635,-0.579303,-13.340153,-1.914109,6.249230,...,9.533860,1.261516,10.305559,4.640141,-14.074480,-4.207365,-4.733542,-6.020726,sub-87964717,SMC
1,12.085660,4.356699,28.328072,12.153649,-5.301539,0.243565,1.138003,-15.566282,0.656770,7.946742,...,4.530734,3.855556,4.987305,6.120726,-16.372959,0.844042,-3.338256,0.795199,sub-87964717,SMC
2,13.301348,9.564412,24.347134,15.509049,-6.083904,1.277819,8.083016,-14.531273,6.095923,4.162280,...,10.417470,3.064260,8.926205,4.880754,-7.483472,3.785343,1.050205,-4.152007,sub-87964717,SMC


quick SVM model
[[51  0 28 25  4]
 [ 4 39 14 49  2]
 [ 8 20 23 27 30]
 [24 15 43 20  6]
 [ 5 18 36  1 48]]
              precision    recall  f1-score   support

        ADHD       0.55      0.47      0.51       108
     HEALTHY       0.42      0.36      0.39       108
         MDD       0.16      0.21      0.18       108
         OCD       0.16      0.19      0.17       108
         SMC       0.53      0.44      0.48       108

    accuracy                           0.34       540
   macro avg       0.37      0.34      0.35       540
weighted avg       0.37      0.34      0.35       540

0.34826024217328566

quick random forest model
[[59  1 27 19  2]
 [ 2 54 17 35  0]
 [ 8 29 24 26 21]
 [18 23 34 24  9]
 [ 0 11 52  1 44]]
              precision    recall  f1-score   support

        ADHD       0.68      0.55      0.61       108
     HEALTHY       0.46      0.50      0.48       108
         MDD       0.16      0.22      0.18       108
         OCD       0.23      0.22      0.23      

#### 3d. 100 length

In [13]:
# best model checkpoint
get_ssl_features('100kernel_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet',
                 conv_length=100)

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,1.077131,-0.604540,2.912446,0.840417,0.615839,0.364553,-2.362027,3.217871,2.688313,1.590816,...,1.402834,0.540021,-0.250136,-0.715285,0.676358,-0.642013,-0.584005,1.288022,sub-87964717,SMC
1,1.438302,1.255031,2.152972,-0.521630,0.730143,0.819524,-1.600391,3.241678,1.814969,1.898302,...,1.110982,0.627550,-1.755466,-0.832604,0.562473,0.106860,-1.540887,0.973580,sub-87964717,SMC
2,1.372186,-1.189366,1.820855,-0.124134,0.425525,0.308911,-1.780870,2.748792,1.599959,1.451993,...,0.495843,0.113271,0.431135,-0.808003,1.167518,0.920891,-1.337937,-0.400199,sub-87964717,SMC


quick SVM model
[[60 12 15 18  3]
 [ 0 46  9 34 19]
 [ 0 30  7 20 51]
 [ 2 42 30  9 25]
 [ 4 14 35  2 53]]
              precision    recall  f1-score   support

        ADHD       0.91      0.56      0.69       108
     HEALTHY       0.32      0.43      0.37       108
         MDD       0.07      0.06      0.07       108
         OCD       0.11      0.08      0.09       108
         SMC       0.35      0.49      0.41       108

    accuracy                           0.32       540
   macro avg       0.35      0.32      0.33       540
weighted avg       0.35      0.32      0.33       540

0.32537384708725897

quick random forest model
[[51 24 12 16  5]
 [ 7 38 19 38  6]
 [ 9 24 24 17 34]
 [14 31 33 21  9]
 [ 8 16 34  9 41]]
              precision    recall  f1-score   support

        ADHD       0.57      0.47      0.52       108
     HEALTHY       0.29      0.35      0.32       108
         MDD       0.20      0.22      0.21       108
         OCD       0.21      0.19      0.20      

In [14]:
# overtrained model
get_ssl_features('fullytrained_100kernel_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet',
                 conv_length=100)

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,2.181792,-5.239806,2.513726,0.312281,4.498333,-3.079380,-4.669401,6.178087,12.772546,6.040983,...,1.479480,-2.767945,3.337354,-3.377693,-10.402101,3.409442,1.267618,-4.279212,sub-87964717,SMC
1,-0.245454,4.698801,2.561338,-0.584578,8.622216,-0.798460,-3.510191,3.621755,10.195595,4.902934,...,2.226113,-3.812922,0.580942,-5.008802,-11.918736,7.577675,-3.059652,-1.727726,sub-87964717,SMC
2,-0.977150,-4.307165,3.619708,-0.777549,7.667782,-3.549263,-6.186962,2.828923,11.079348,5.299800,...,2.277834,-5.642869,3.306869,-5.687233,-6.840755,6.385839,-1.838228,-1.185866,sub-87964717,SMC


quick SVM model
[[58  0 25 18  7]
 [ 4 57 11 34  2]
 [14 19 21 37 17]
 [30 15 34 26  3]
 [ 5 21 44  0 38]]
              precision    recall  f1-score   support

        ADHD       0.52      0.54      0.53       108
     HEALTHY       0.51      0.53      0.52       108
         MDD       0.16      0.19      0.17       108
         OCD       0.23      0.24      0.23       108
         SMC       0.57      0.35      0.43       108

    accuracy                           0.37       540
   macro avg       0.40      0.37      0.38       540
weighted avg       0.40      0.37      0.38       540

0.3776342520878836

quick random forest model
[[62  1 27 14  4]
 [ 7 58 11 32  0]
 [ 5 26 20 35 22]
 [23 22 37 17  9]
 [ 4 15 42  4 43]]
              precision    recall  f1-score   support

        ADHD       0.61      0.57      0.59       108
     HEALTHY       0.48      0.54      0.50       108
         MDD       0.15      0.19      0.16       108
         OCD       0.17      0.16      0.16       

#### 3e. 64 length 16 filters

In [6]:
# best model checkpoint
get_ssl_features('16-64kernel_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet',
                 conv_length=64, n_filters=16)

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,-0.043707,-0.530889,2.721297,-0.527805,-0.999529,0.160205,-0.788692,-0.824401,1.580407,0.454592,...,0.301767,-2.072564,-2.132128,-0.155470,0.395713,1.659627,0.024428,-0.477699,sub-87964717,SMC
1,-0.340989,-1.125356,1.987100,-0.189399,-0.325343,0.494679,-1.692619,-1.973698,1.755913,1.005224,...,1.094240,-0.047524,-1.000838,-0.443625,1.703377,1.154400,-0.268609,-0.796578,sub-87964717,SMC
2,-0.164598,-0.689947,1.602566,0.488093,-1.000122,-0.787271,-1.303537,-1.311452,1.937030,0.691065,...,0.698513,-2.140108,-1.115033,0.238866,1.133382,1.536821,0.187970,-0.348470,sub-87964717,SMC


quick SVM model
[[59  1 15 31  2]
 [ 4 31  6 49 18]
 [ 0 11  7 56 34]
 [ 2 10 29 24 43]
 [ 0 18 35 18 37]]
              precision    recall  f1-score   support

        ADHD       0.91      0.55      0.68       108
     HEALTHY       0.44      0.29      0.35       108
         MDD       0.08      0.06      0.07       108
         OCD       0.13      0.22      0.17       108
         SMC       0.28      0.34      0.31       108

    accuracy                           0.29       540
   macro avg       0.37      0.29      0.31       540
weighted avg       0.37      0.29      0.31       540

0.31441338634768

quick random forest model
[[61  3 21 19  4]
 [12 43 17 26 10]
 [ 8 15 26 36 23]
 [ 9 14 43 24 18]
 [ 6 17 24 33 28]]
              precision    recall  f1-score   support

        ADHD       0.64      0.56      0.60       108
     HEALTHY       0.47      0.40      0.43       108
         MDD       0.20      0.24      0.22       108
         OCD       0.17      0.22      0.20       10

In [7]:
# overtrained model
get_ssl_features('fullytrained_16-64kernel_acrossSub_ShallowNet_pretext_model', dataset, df_sample, eval=True, to_disk=True, pretext_model='ShallowNet',
                 conv_length=64, n_filters=16)

features_df.shape = (2700, 102)


,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,ID,diagnosis
0,0.746049,-16.331999,9.022522,-4.335643,0.054456,0.672561,-10.877899,-7.041751,13.345524,18.684858,...,3.370183,2.704486,4.610126,-1.819774,-3.713675,-15.618497,1.562948,-4.381397,sub-87964717,SMC
1,0.317036,-18.856760,6.844414,-3.837267,-3.774901,1.411035,-11.496122,-4.431200,18.649862,13.299951,...,1.517679,0.620139,10.263021,-1.664523,-6.825373,-15.260834,0.103055,-2.406564,sub-87964717,SMC
2,0.612150,-17.843058,7.398017,-2.570997,-0.309375,1.083591,-10.568644,-6.399098,9.103529,14.098498,...,0.081484,-0.229301,3.375617,1.767349,-6.376538,-15.032051,1.747023,-0.163190,sub-87964717,SMC


quick SVM model
[[57  0 34 12  5]
 [14 43 15 31  5]
 [10 36 24 22 16]
 [34 12 48 10  4]
 [ 2 15 37  7 47]]
              precision    recall  f1-score   support

        ADHD       0.49      0.53      0.51       108
     HEALTHY       0.41      0.40      0.40       108
         MDD       0.15      0.22      0.18       108
         OCD       0.12      0.09      0.11       108
         SMC       0.61      0.44      0.51       108

    accuracy                           0.34       540
   macro avg       0.36      0.34      0.34       540
weighted avg       0.36      0.34      0.34       540

0.34047164387351303

quick random forest model
[[62  7 19 19  1]
 [10 49  9 36  4]
 [ 6 36 23 22 21]
 [27 12 44 13 12]
 [ 1 16 46  8 37]]
              precision    recall  f1-score   support

        ADHD       0.58      0.57      0.58       108
     HEALTHY       0.41      0.45      0.43       108
         MDD       0.16      0.21      0.18       108
         OCD       0.13      0.12      0.13      